In [ ]:
import numpy as np
import pandas as pd
import torch

import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.ODE_Transformer_Dataset import ODE_Transformer_Dataset

from Model.SelfAttention import Self_attention, Attention_LSTM
from Model.LSTM import LSTM
from Model.ODERNN import ODE_RNN

from Model.ODE_Attention import ODE_Attention
from utils import preprocess

In [ ]:
max_size = 311
window_size = 5
hidden_size =256

batch_size = 1

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

transaction_df = pd.read_csv('../데이터/Transaction/transaction_all.csv')
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')

transaction_df, economy_df = preprocess(transaction_df, economy_df, window_size=5)

In [ ]:
# gpu 연결
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is available')

# checkpoint 받기
lstm_model = LSTM(input_size=5, hidden_size=hidden_size, output_size=output_size, device=device)
lstm_model.load_state_dict(torch.load("../데이터/checkpoint/best_rnn_model.pth"))

############## cpu?
tran_ODE = ODE_RNN(256, 64, batch_size, 200,200, device).to(device)
tran_ODE.load_state_dict(torch.load("../데이터/checkpoint/best_ODE_transaction_model.pth",map_location=torch.device('cpu')))